# Define Event

Before we can analyze the event, we need to identify the boundaries of the event. We will accomplish this by

1. Defining the values to be classified as gains and losses
2. Creating a counts table defining the various events
3. Defining the porportion of patients with gain or loss to be considered significant
4. Identifying regions of gain and loss
5. Identifying regions where all cancer types meet criteria for gain or loss event.

## Setup

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import cptac
import numpy as np
import cnvutils

cptac warning: Your version of cptac (0.8.5) is out-of-date. Latest is 0.8.6. Please run 'pip install --upgrade cptac' to update it. (C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py, line 4)


## Part 1: Define Parameters

These are the parameters that must be set for each analysis.

In [2]:
# # A list of cancer types to look at (should be all capital letter ex: BRCA)
# CANCER_TYPES = ['BRCA', 'COLON', 'HNSCC', 'LSCC', 'LUAD', 'OVARIAN']
# The porportion of patients with gain or loss of a given gene to be considered
PATIENT_CUTOFF = 0.2
# The chromosome to be analyzed (should be a string)
CHROMOSOME = '20'
# The arm to be analyzed. Options are: 'p', 'q' or 'both'.
ARM = 'both'

In [3]:
# Here you will need to load in all the cancer types to be considered. We have listed all for convenience, simply comment
# out the cancer types you do not want to consider.
cnv = {
    'BRCA': cptac.Brca().get_CNV(),
    'CCRCC': cptac.Ccrcc().get_CNV(),
    'COLON': cptac.Colon().get_CNV(),
    'ENDO': cptac.Endometrial().get_CNV(),
    'GBM': cptac.Gbm().get_CNV(),
    'HNSCC': cptac.Hnscc().get_CNV(),
    'LSCC': cptac.Lscc().get_CNV(),
    'LUAD': cptac.Luad().get_CNV(),
    'OVARIAN': cptac.Ovarian().get_CNV()
}

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py, line 8)


cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py, line 9)


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py, line 10)


## Part 2: Determine Cutoffs

In [4]:
counts = cnvutils.get_counts_table()

C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
cutoffs = dict()
for cancer_type in cnv.keys():
    df = cnv[cancer_type]
    cutoffs[cancer_type] = len(df) * PATIENT_CUTOFF

## Part 3: Subset Event

In [6]:
# counts.cancer.unique()

In [7]:
counts = counts[(counts.chromosome == CHROMOSOME)]
if ARM != 'both':
    counts = counts[counts.arm == ARM]

In [8]:
# counts

## Find Loss Regions

In [9]:
df = counts
loss_event_locations = dict()
for cancer in cnv.keys():
    df_loss = df[(df.variable == 'loss') & (df.cancer == cancer)].sort_values('start_bp')
    print(len(df_loss))
    values = list(df_loss.value)
    loss_events = list()
    start = None
    for i in range(0, len(values)):
        val = values[i]
        if val > cutoffs[cancer]:
            if start is None:
                start = i
        else:
            if start is not None:
                loss_events.append((start, i))
                start = None
    if start is not None:
        loss_events.append((start, len(values)-1))
    event_locations = list()
    for event in loss_events:
        start_bp = df_loss.iloc[event[0]].start_bp
        end_bp = df_loss.iloc[event[1]].start_bp
        event_locations.append((start_bp, end_bp-start_bp))
    loss_event_locations[cancer] = event_locations

645
514
614
723
540
1398
675
514
495


In [10]:
loss_event_locations['LUAD']

[]

In [11]:
loss_event_patients = list()
for cancer in loss_event_locations.keys():
    events = loss_event_locations[cancer]
    for event in events:
        start = event[0]
        end = event[0] + event[1]
        loss_event_patients.append((start, 1))
        loss_event_patients.append((end, 0))
#     patients += list(gain_event_locations[cancer])
loss_event_patients.sort()

In [12]:
counts

,Name,Database_ID,start_bp,end_bp,cancer,chromosome,arm,variable,value
31,AAR2,ENSG00000131043.7,36236459.0,36270918.0,BRCA,20,q,gain,45
98,ABHD12,ENSG00000100997.14,25294742.0,25390835.0,BRCA,20,p,gain,34
107,ABHD16B,ENSG00000183260.5,63861498.0,63862988.0,BRCA,20,q,gain,47
202,ACOT8,ENSG00000101473.12,45841721.0,45857405.0,BRCA,20,q,gain,46
232,ACSS1,ENSG00000154930.10,25006230.0,25058980.0,BRCA,20,p,gain,39
...,...,...,...,...,...,...,...,...,...
476428,Z98752.1,NaN,43549389.0,43550949.0,HNSCC,20,q,loss,4
476429,Z98752.2,NaN,43523261.0,43523572.0,HNSCC,20,q,loss,4
476430,Z98752.4,NaN,43590009.0,43590596.0,HNSCC,20,q,loss,4
476555,ZNF840P,NaN,46484461.0,46492640.0,HNSCC,20,q,loss,4


In [13]:
count = 0
current_bp = 0
start = list()
end = list()
size = list()
total = list()
for patient in loss_event_patients:
    if patient[0] != current_bp:
        start.append(current_bp)
        end.append(patient[0])
        size.append(patient[0]-current_bp)
        total.append(count)
        current_bp = patient[0]
    if patient[1] == 1:
        count += 1
    else:
        count -= 1
loss_event_data = pd.DataFrame({'start': start, 'end': end, 'counts': total, 'length': size})

In [14]:
loss_event_data

,start,end,counts,length
0,0.0,87250.0,0,87250.0
1,87250.0,869900.0,1,782650.0
2,869900.0,2101827.0,0,1231927.0
3,2101827.0,2655067.0,1,553240.0
4,2655067.0,2656624.0,1,1557.0
5,2656624.0,6446723.0,1,3790099.0
6,6446723.0,13389392.0,0,6942669.0
7,13389392.0,14322985.0,1,933593.0
8,14322985.0,14884250.0,0,561265.0
9,14884250.0,16272104.0,1,1387854.0


## Find Gain Regions

In [15]:
df = counts
gain_event_locations = dict()
for cancer in cnv.keys():
    df_gain = df[(df.variable == 'gain') & (df.cancer == cancer)].sort_values('start_bp')
    values = list(df_gain.value)
    gain_events = list()
    start = None
    for i in range(0, len(values)):
        val = values[i]
        if val > cutoffs[cancer]:
            if start is None:
                start = i
        else:
            if start is not None:
                gain_events.append((start, i))
                start = None
    if start is not None:
        gain_events.append((start, len(values)-1))
    event_locations = list()
    for event in gain_events:
        start_bp = df_gain.iloc[event[0]].start_bp
        end_bp = df_gain.iloc[event[1]].start_bp
        
        event_locations.append((start_bp, end_bp-start_bp))
    gain_event_locations[cancer] = event_locations

In [16]:
gain_event_patients = list()
for cancer in gain_event_locations.keys():
    events = gain_event_locations[cancer]
    for event in events:
        start = event[0]
        end = event[0] + event[1]
        gain_event_patients.append((start, 1))
        gain_event_patients.append((end, 0))
#     patients += list(gain_event_locations[cancer])
gain_event_patients.sort()

In [17]:
count = 0
current_bp = 0
# results = list()
start = list()
end = list()
size = list()
total = list()
for patient in gain_event_patients:
    if patient[0] != current_bp:
        start.append(current_bp)
        end.append(patient[0])
        size.append(patient[0]-current_bp)
        total.append(count)
#         results.append((current_bp, patient[0], count))
        current_bp = patient[0]
    if patient[1] == 1:
        count += 1
    else:
        count -= 1
gain_event_data = pd.DataFrame({'start': start, 'end': end, 'counts': total, 'length': size})

In [18]:
gain_event_data

,start,end,counts,length
0,0.0,87250.0,0,87250.0
1,87250.0,2655067.0,5,2567817.0
2,2655067.0,2656097.0,4,1030.0
3,2656097.0,2656624.0,5,527.0
4,2656624.0,31563166.0,5,28906542.0
5,31563166.0,34516409.0,6,2953243.0
6,34516409.0,35002404.0,5,485995.0
7,35002404.0,35030317.0,6,27913.0
8,35030317.0,35115364.0,5,85047.0
9,35115364.0,35195306.0,6,79942.0
